# Transfer Learning

### Image Mode

In [16]:
IMAGE_MODE = 1      #1 for colour, 0 for grayscale, unchanged for -1

### Datasets locations:

In [17]:
import os

height = 100
width  = 100

data_dir = os.path.join(os.path.pardir, "Datasets", "Resized_data_{}_{}".format(height, width))
train_csv = os.path.join(data_dir, "train.csv")
test_csv = os.path.join(data_dir, "test.csv")

N_CLASSES = 2

## Imports

In [18]:
import tensorflow as tf

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cv2

## Loading the dataset

In [19]:
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [20]:
train_df.head()

,Image_Path,Parasitized
0,..\Datasets\Resized_data_100_100\Parasitized\1...,1.0
1,..\Datasets\Resized_data_100_100\Parasitized\1...,1.0
2,..\Datasets\Resized_data_100_100\Uninfected\10...,0.0
3,..\Datasets\Resized_data_100_100\Parasitized\1...,1.0
4,..\Datasets\Resized_data_100_100\Parasitized\1...,1.0


In [21]:
train_df.dtypes

Image_Path      object
Parasitized    float64
dtype: object

## Data and Labels

In [22]:
train_x = train_df['Image_Path'].to_numpy()
train_y = train_df['Parasitized'].to_numpy()

test_x  = test_df['Image_Path'].to_numpy()
test_y  = test_df['Parasitized'].to_numpy()

#### Load images from paths

In [23]:
def load_images(path_arr):
    '''Reads and loads images into a numpy array
    Returns: a numpy array'''
    arr = []
    for path in path_arr:
        arr.append(cv2.imread(path, IMAGE_MODE))
    
    return np.array(arr)


In [24]:
train_x = load_images(train_x)
test_x = load_images(test_x)

#### Checking shape

In [25]:
print(train_x.shape, test_x.shape, sep = '\n')

(24802, 100, 100, 3)
(2756, 100, 100, 3)


## DNN

In [26]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [27]:
# batch sizes
TRAIN_BATCH_SIZE = 64
TEST_BATCH_SIZE  = 32

In [28]:
train_x = train_x/255.0
test_x = test_x/255.0

train_gen = DataGenerator(train_x, train_y, TRAIN_BATCH_SIZE)
test_gen  = DataGenerator(test_x, test_y, TEST_BATCH_SIZE)


In [29]:
height, width = train_x.shape[1], train_x.shape[2]
dims = len(train_x.shape)-1

In [30]:
input_shape = list(train_x.shape[1:])
if(len(input_shape) == 2): input_shape.append(1)

input_shape

[100, 100, 3]

In [40]:
imagenet = tf.keras.applications.Xception(
    weights = 'imagenet',
    include_top = False,
    input_shape = input_shape
)
imagenet.trainable = False

dnn_model = tf.keras.Sequential([
    imagenet,

    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(N_CLASSES, activation = 'sigmoid')
])

dnn_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.003),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

history = dnn_model.fit(train_gen,
                shuffle = True,
                epochs = 50,
                validation_data = test_gen, 
                callbacks = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3))

Epoch 1/50
388/388 [==============================] - 14s 33ms/step - loss: 0.2613 - accuracy: 0.9019 - val_loss: 0.2185 - val_accuracy: 0.9187
Epoch 2/50
388/388 [==============================] - 12s 32ms/step - loss: 0.1943 - accuracy: 0.9224 - val_loss: 0.2125 - val_accuracy: 0.9194
Epoch 3/50
388/388 [==============================] - 12s 32ms/step - loss: 0.1813 - accuracy: 0.9284 - val_loss: 0.1888 - val_accuracy: 0.9253
Epoch 4/50
388/388 [==============================] - 12s 32ms/step - loss: 0.1734 - accuracy: 0.9320 - val_loss: 0.1949 - val_accuracy: 0.9260
Epoch 5/50
388/388 [==============================] - 12s 32ms/step - loss: 0.1640 - accuracy: 0.9351 - val_loss: 0.1946 - val_accuracy: 0.9260
Epoch 6/50
388/388 [==============================] - 12s 32ms/step - loss: 0.1572 - accuracy: 0.9393 - val_loss: 0.2014 - val_accuracy: 0.9267
